In [9]:

import os
import glob
from pathlib import Path

import numpy as np
from tqdm import tqdm



base_folder_path = Path('/data/storage025/BoundaryTone_single_channel_normalized_nosil/')

np.set_printoptions(threshold=np.inf)


# 3 sublists for YA OA PD
exp2lists = {'BoundaryTone': [[], [], []], 'EarlyLate': [[], [], []], 'PictureNaming': [[], [], []]}
all3 = [[], [], []]


def get_group_id(filename):
    filename = os.path.basename(filename)
    group_id = filename.split('_')[0][-4:-2]
    if group_id not in ['11', '21', '22']:
        raise ValueError(f"Invalid group id {group_id}")
    return group_id

def add2list(group_id, feature, ls):
    if group_id == '11':
        ls[0].append(feature)
    elif group_id == '21':
        ls[1].append(feature)
    elif group_id == '22':
        ls[2].append(feature)
    else:
        print(f'Invalid group id {group_id}')
        
        
        
feature_name = 'f0'        
avg_diff = []
f0_diff_list = open('f0_diff_list.txt', 'w')
non_f0_list = open('non_f0_list.txt', 'w')
# for folder in ['BoundaryTone', 'EarlyLate', 'PictureNaming']:
for folder in ['BoundaryTone']:
    feature_folder = os.path.join(base_folder_path, folder + '-features', feature_name)
    feature_folder = Path(feature_folder)
    npy_files = list(feature_folder.glob('*.npy'))
    print(f'Processing {folder} folder...')
    print(f'Found {len(npy_files)} npy files')
    

    cnt = 0
    cnt2 = 0
    for npy_file in npy_files:
        feature = np.load(npy_file)
        feature_nonorm = np.load(str(npy_file).replace('_normalized_', '_'))
        if len(feature) != len(feature_nonorm):
            print(f'Length mismatch in {npy_file}')
            f0_diff_list.write(f'{os.path.basename(npy_file)}\t{None}\n') 
            continue
        else:
            avg_diff_each = np.average(np.abs(feature - feature_nonorm))
        if avg_diff_each > 1:
            f0_diff_list.write(f'{os.path.basename(npy_file)}\t{avg_diff_each}\n') 
        avg_diff.append(avg_diff_each)
        
        # check if all 0 value
        if np.max(feature) == 0 and np.min(feature) == 0:
            cnt += 1
            non_f0_list.write(f'{os.path.basename(npy_file)}\n')
            continue
        
        if np.max(feature) < 60:
            print(f'max value < 60 in {npy_file}')
            cnt2 += 1
            non_f0_list.write(f'{os.path.basename(npy_file)}\n')

            continue
        
        group_id = get_group_id(npy_file.stem)
        add2list(group_id, feature, exp2lists[folder])
    print(f'{cnt} files with all 0 values')
    print(f'{cnt2} files with max value < 60Hz')


#merge 3lists to all3
for i in range(3):
    all3[i] += exp2lists['BoundaryTone'][i] + exp2lists['EarlyLate'][i] + exp2lists['PictureNaming'][i]

avg_diff = np.array(avg_diff)
# print top 10 largest diff
print(len(avg_diff))
# print(np.sort(avg_diff)[-800])
print(np.min(avg_diff))
print(np.average(avg_diff)) 
# close
f0_diff_list.close()
non_f0_list.close()
        


Processing BoundaryTone folder...
Found 5032 npy files
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-1101_schroef_2obj_mbt_men.wav_1.wav_f0.npy
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-1108_bowlingbal_1obj_mbt_zen.wav_1.wav_f0.npy
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-1117_paraplu_1obj_mbt_zen.wav_1.wav_f0.npy
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-2117_bowlingbal_1obj_mbt_zen.wav_1.wav_f0.npy
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-2120_punaise_2obj_zbt_zen.wav_1.wav_f0.npy
Length mismatch in /data/storage025/BoundaryTone_single_channel_normalized_nosil/BoundaryTone-features/f0/subj-2204_triangel_2obj_mbt_men.wav_1.wav_f0.npy
Length mismatc